<a href="https://colab.research.google.com/github/nandini-gangrade/Data-Engineering-Hexaware/blob/Project/Week4/Azure_Databricks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 1: Data Warehousing and SQL Concepts

In [1]:
# Step 1: Set up SQLite in Colab
import sqlite3

# Create a new SQLite database (in-memory)
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Step 2: Create tables to store customer, product, and order data
cursor.execute('''
CREATE TABLE customer_dim (
    customer_id INT PRIMARY KEY,
    customer_name VARCHAR(255),
    location VARCHAR(255)
);
''')

cursor.execute('''
CREATE TABLE product_dim (
    product_id INT PRIMARY KEY,
    product_name VARCHAR(255),
    category VARCHAR(255)
);
''')

cursor.execute('''
CREATE TABLE order_fact (
    order_id INT PRIMARY KEY,
    product_id INT,
    customer_id INT,
    quantity INT,
    order_amount DECIMAL(10, 2),
    order_date TIMESTAMP
);
''')

# Step 3: Insert sample data into the tables
cursor.execute("INSERT INTO product_dim VALUES (1, 'Smartphone', 'Electronics')")
cursor.execute("INSERT INTO product_dim VALUES (2, 'Laptop', 'Electronics')")
cursor.execute("INSERT INTO product_dim VALUES (3, 'Tablet', 'Electronics')")

cursor.execute("INSERT INTO customer_dim VALUES (2001, 'Bob', 'California')")
cursor.execute("INSERT INTO customer_dim VALUES (2002, 'Alice', 'Texas')")
cursor.execute("INSERT INTO customer_dim VALUES (2003, 'John', 'New York')")

cursor.execute("INSERT INTO order_fact VALUES (1001, 1, 2001, 1, 299.99, '2023-09-27 12:30:00')")
cursor.execute("INSERT INTO order_fact VALUES (1002, 2, 2002, 2, 999.99, '2023-09-27 14:45:00')")
cursor.execute("INSERT INTO order_fact VALUES (1003, 3, 2003, 1, 499.99, '2023-09-27 15:30:00')")

# Commit the changes
conn.commit()

# Step 4: Query data to analyze customer behavior and product interactions
# 1. Retrieve all customer data
cursor.execute("SELECT * FROM customer_dim")
customers = cursor.fetchall()
print("Customer Data:")
for row in customers:
    print(row)

# 2. Retrieve all product data
cursor.execute("SELECT * FROM product_dim")
products = cursor.fetchall()
print("\nProduct Data:")
for row in products:
    print(row)

# 3. Retrieve all order data
cursor.execute("SELECT * FROM order_fact")
orders = cursor.fetchall()
print("\nOrder Data:")
for row in orders:
    print(row)

# 4. Analyze total sales per product
cursor.execute('''
SELECT product_dim.product_name, SUM(order_fact.order_amount) AS total_sales
FROM order_fact
JOIN product_dim ON order_fact.product_id = product_dim.product_id
GROUP BY product_dim.product_name
''')
sales_analysis = cursor.fetchall()
print("\nTotal Sales per Product:")
for row in sales_analysis:
    print(row)

# Close the connection
conn.close()


Customer Data:
(2001, 'Bob', 'California')
(2002, 'Alice', 'Texas')
(2003, 'John', 'New York')

Product Data:
(1, 'Smartphone', 'Electronics')
(2, 'Laptop', 'Electronics')
(3, 'Tablet', 'Electronics')

Order Data:
(1001, 1, 2001, 1, 299.99, '2023-09-27 12:30:00')
(1002, 2, 2002, 2, 999.99, '2023-09-27 14:45:00')
(1003, 3, 2003, 1, 499.99, '2023-09-27 15:30:00')

Total Sales per Product:
('Laptop', 999.99)
('Smartphone', 299.99)
('Tablet', 499.99)


# Week 2: Real-Time Data Processing with Python

In [ ]:
# Install the necessary libraries in Colab
!pip install sqlalchemy pandas

In [6]:
import pandas as pd
from sqlalchemy import create_engine
import random
from datetime import datetime
import time

In [7]:
# Step 2.1: Create a connection to SQLite database
# This creates a file 'ecommerce.db' in your Colab environment
engine = create_engine('sqlite:///ecommerce.db')

In [12]:
# Step 3: Function to simulate real-time data streams
def simulate_order_stream():
    product_ids = [1, 2, 3, 4]  # Product IDs available in the system
    customer_ids = [2001, 2002, 2003]  # Customer IDs available in the system
    batch_count = 0  # Initialize a counter for batch number
    max_batches = 5  # Maximum number of batches to simulate

    while batch_count < max_batches:
        # Create a random order
        order = {
            'order_id': random.randint(1001, 9999),
            'product_id': random.choice(product_ids),
            'customer_id': random.choice(customer_ids),
            'quantity': random.randint(1, 5),
            'order_amount': round(random.uniform(50, 500), 2),
            'order_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }

        # Yield the order as a Pandas DataFrame
        yield pd.DataFrame([order])

        # Increment the batch count
        batch_count += 1

        # Sleep for a short time to simulate real-time data
        time.sleep(2)

In [13]:
# Step 4: Stream and load real-time data into the SQL table
# Run this cell to start the data streaming process
for order_batch in simulate_order_stream():
    # Append the generated order batch to the order_fact table
    order_batch.to_sql('order_fact', con=engine, if_exists='append', index=False)

    # Print only a summary of each order batch with the timestamp of loading
    print(f"Order batch loaded at {datetime.now()} - Order ID: {order_batch['order_id'].iloc[0]}, "
          f"Product ID: {order_batch['product_id'].iloc[0]}, Customer ID: {order_batch['customer_id'].iloc[0]}")

Order batch loaded at 2024-09-27 08:20:14.097668 - Order ID: 2849, Product ID: 2, Customer ID: 2001
Order batch loaded at 2024-09-27 08:20:16.113633 - Order ID: 6997, Product ID: 4, Customer ID: 2002
Order batch loaded at 2024-09-27 08:20:18.131926 - Order ID: 8431, Product ID: 4, Customer ID: 2003
Order batch loaded at 2024-09-27 08:20:20.148154 - Order ID: 1193, Product ID: 2, Customer ID: 2003
Order batch loaded at 2024-09-27 08:20:22.164712 - Order ID: 2906, Product ID: 1, Customer ID: 2001


#  Week 3: Real-Time Data Processing with Apache Spark and PySpark

In [ ]:
# Step 1
# Set Up Spark
!pip install pyspark

In [22]:
# Import Necessary Libraries

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, TimestampType
from pyspark.sql.functions import col, sum, count
import pandas as pd
import random
import os
import time
from datetime import datetime

In [16]:
# Step 2
# Create Spark session
spark = SparkSession.builder \
    .appName("EcommerceRealTime") \
    .getOrCreate()

In [48]:
# Step 3: Create a directory for streaming data
streaming_data_path = os.makedirs('/content/streaming_data', exist_ok=True)

# Step 4: Define schema for order data
schema = StructType([
    StructField("order_id", IntegerType(), True),
    StructField("product_id", IntegerType(), True),
    StructField("customer_id", IntegerType(), True),
    StructField("quantity", IntegerType(), True),
    StructField("order_amount", FloatType(), True),
    StructField("order_date", TimestampType(), True)
])

# Step 5: Simulate initial streaming data with new values
initial_order_data = [
    (2001, 1, 3001, 2, 149.99, datetime.now()),  # Order 1
    (2002, 2, 3002, 3, 199.99, datetime.now()),  # Order 2
    (2003, 3, 3003, 1, 299.99, datetime.now()),  # Order 3
    (2004, 4, 3001, 4, 399.99, datetime.now()),  # Order 4
    (2005, 5, 3002, 2, 599.99, datetime.now()),  # Order 5
]

# Create a DataFrame from the list of initial orders
df_orders = spark.createDataFrame(initial_order_data, schema)

# Show the initial data
print("Initial Orders:")
df_orders.show()

# Step 6: Process Data Using PySpark
# Group the data by product_id and calculate the total sales (sum of order_amount) per product
product_sales = df_orders.groupBy("product_id").agg(
    sum("order_amount").alias("total_sales"),
    count("order_id").alias("order_count")
)

# Show the result
print("Initial Product Sales:")
product_sales.show()

# Step 7: Real-Time Streaming Simulation
# Simulating appending new data in real-time with updated values
new_order_data = [
    (1008, 2, 3004, 2, 249.99, datetime.now()),  # New Order 1
    (1009, 4, 3001, 3, 359.95, datetime.now()),  # New Order 2
    (1010, 5, 3002, 1, 599.99, datetime.now()),  # New Order 3
    (1011, 1, 3003, 4, 149.99, datetime.now()),  # New Order 4
    (1012, 3, 3004, 2, 389.50, datetime.now()),  # New Order 5
]

# Create a new DataFrame for the new batch of orders with the correct timestamp format
new_df_orders = spark.createDataFrame(new_order_data, schema)

# Append the new data to the original DataFrame
df_orders = df_orders.union(new_df_orders)

# Perform the same aggregation again with the updated data
updated_product_sales = df_orders.groupBy("product_id").agg(
    sum("order_amount").alias("total_sales"),
    count("order_id").alias("order_count")
)

# Show the updated result
print("Updated Product Sales After New Orders:")
updated_product_sales.show()

# Step 8: Write the results to a CSV file
updated_product_sales.write.csv("/content/product_sales.csv", header=True, mode='overwrite')

Initial Orders:
+--------+----------+-----------+--------+------------+--------------------+
|order_id|product_id|customer_id|quantity|order_amount|          order_date|
+--------+----------+-----------+--------+------------+--------------------+
|    2001|         1|       3001|       2|      149.99|2024-09-27 09:45:...|
|    2002|         2|       3002|       3|      199.99|2024-09-27 09:45:...|
|    2003|         3|       3003|       1|      299.99|2024-09-27 09:45:...|
|    2004|         4|       3001|       4|      399.99|2024-09-27 09:45:...|
|    2005|         5|       3002|       2|      599.99|2024-09-27 09:45:...|
+--------+----------+-----------+--------+------------+--------------------+

Initial Product Sales:
+----------+------------------+-----------+
|product_id|       total_sales|order_count|
+----------+------------------+-----------+
|         1|149.99000549316406|          1|
|         2|199.99000549316406|          1|
|         3|  299.989990234375|          1|
|  

 # Week 4: ETL Pipelines in Azure Databricks

In [ ]:
!pip install pyspark
!pip install delta-spark

In [53]:
# Import necessary libraries
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder \
    .appName("DeltaLakeETL") \
    .getOrCreate()

In [ ]:
# Create a DataFrame for customer data
customer_data = [
    (2001, 'Shruti', 'India'),
    (2002, 'Priyanshi', 'Canada'),
    (2003, 'Prakhar', 'New York'),
    (2004, 'Nandini', 'USA'),
    (2005, 'Divya', 'Australia')
]

customer_columns = ["customer_id", "customer_name", "location"]
customer_df = spark.createDataFrame(customer_data, customer_columns)

# Write the DataFrame to Delta format
customer_df.write.format("delta").mode("overwrite").save("/content/delta/customer_dim")

# Create a DataFrame for order data
order_data = [
    (1001, 1, 2001, 1, 139.99, '2023-09-27 12:30:00'),
    (1002, 2, 2002, 2, 934.99, '2023-09-27 14:45:00'),
    (1003, 3, 2003, 1, 543.99, '2023-09-27 15:30:00'),
    (1004, 1, 2004, 2, 299.99, '2023-09-27 16:00:00'),
    (1005, 2, 2005, 1, 499.99, '2023-09-27 17:30:00')
]

order_columns = ["order_id", "product_id", "customer_id", "quantity", "order_amount", "order_date"]
order_df = spark.createDataFrame(order_data, order_columns)

# Write the order DataFrame to Delta format
order_df.write.format("delta").mode("overwrite").save("/content/delta/order_fact")

import dlt

@dlt.table
def order_fact():
    return spark.read.format("delta").load("/content/delta/order_fact")

@dlt.table
def customer_dim():
    return spark.read.format("delta").load("/content/delta/customer_dim")

# Show data from Delta Live Tables
df_dltCustomers = spark.read.format("delta").load("/content/delta/customer_dim")
df_dltCustomers.show()

df_dlt_Order_fact = spark.read.format("delta").load("/content/delta/order_fact")
df_dlt_Order_fact.show()
